# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

output_data_file = "../output_data/cities.csv"
city_data = pd.read_csv(output_data_file)
city_data.dropna(inplace = True)
city_data.head()
# city_data_df = pd.DataFrame(city_data, columns=['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country'])


# city_data_df.head(25)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations=city_data[['Lat', 'Lng']].astype(float)

hum_max=city_data['Humidity'].max()



In [4]:
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=hum_max,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:
A max temperature lower than 80 degrees but higher than 70

Wind speed less than 10 mph

Zero cloudiness

In [7]:
weather_min = city_data.loc[city_data['Max Temp']< 80,:]
weather_max = city_data.loc[city_data['Max Temp'] >70,:]
weather_wind = city_data.loc[city_data['Wind Speed'] <10,:] 
weather_cloud = city_data.loc[city_data['Cloudiness'] != 0,:]
weather_cloud.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame()
name_hotel = pd.DataFrame(weather_cloud)
hotel_df['City'] =name_hotel['City']
hotel_df['Country'] = name_hotel['Country']
hotel_df['Lat'] = name_hotel['Lat']
hotel_df['Lng'] = name_hotel['Lng']

hotel_df.head()

,City,Country,Lat,Lng
0,ostrovnoy,RU,68.05,39.51
1,mahebourg,MU,-20.41,57.70
2,qaanaaq,GL,77.48,-69.36
4,cape town,ZA,-33.93,18.42
5,atuona,PF,-9.80,-139.03


In [10]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: ostrovnoy.
Closest hotel to ostrovnoy at 68.05 , 39.51 is Gorodskoy Dom Kul'tury.
------------
Retrieving Results for Index 1: mahebourg.
Closest hotel to mahebourg at -20.41 , 57.7 is Holiday Inn Mauritius Mon Tresor.
------------
Retrieving Results for Index 2: qaanaaq.
Closest hotel to qaanaaq at 77.48 , -69.36 is Qaanaaq Supermarket.
------------
Retrieving Results for Index 4: cape town.
Closest hotel to cape town at -33.93 , 18.42 is The Capital 15 on Orange Hotel.
------------
Retrieving Results for Index 5: atuona.
Closest hotel to atuona at -9.8 , -139.03 is Atuona rent a car.
------------
Retrieving Results for Index 8: tiksi.
Closest hotel to tiksi at 71.64 , 128.87 is Aktsionernoye Obshchestvo "Sakhaenergo".
------------
Retrieving Results for Index 9: pushkinskiye gory.
Closest hotel to pushkinskiye gory at 57.02 , 28.91 is House in Voronich.
------------
Retrieving Results for Index 10: aykhal.
Closest hotel to aykhal at 65.95 , 111.51 is S

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_loc = hotel_df[["Lat", "Lng"]]

In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=hum_max,
                                 dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

TypeError: object of type 'int' has no len()

In [16]:
figure = gmaps.figure()

heat_layer_map = gmaps.heatmap_layer(locations_loc, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations_loc, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig

ValueError: weights must be of the same length as locations or None

In [14]:
# Add marker layer ontop of heat map

figure = gmaps.figure()

# Create heat layer
heat_layer_heat = gmaps.heatmap_layer(locations_loc, weights=humidity, 
                                 dissipating=False, max_intensity=hum_max,
                                 point_radius=1)


# markers = gmaps.marker_layer(locations_loc)
# # Add layer
# figure.add_layer(heat_layer)
# fig.add_layer(markers)

# plt.savefig("vacationheatmapwithhotels.png")



# # Display figure
# fig




ValueError: weights must be of the same length as locations or None